In [62]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user

In [63]:
data_file=r'D:\PythonDev\MyQuantFinProject\Data\4ETF-MarToSep23-Funds.csv'
isPy=False
if os.path.exists(data_file):
 print(f"Found {os.path.abspath(data_file)}")
else:
 raise FileNotFoundError(f"Not found file {data_file}")  

Found D:\PythonDev\MyQuantFinProject\Data\4ETF-MarToSep23-Funds.csv


# Specify Period and Timefame to caculate fund performance and

In [64]:
from_str='2023-05-10'
to_str='2023-09-30' 
timeframe='M'
#timeframe='W-FRI'

# Input data

In [65]:
if isPy:
    print(f"Data csv file in {data_file}")
    data_file=input("CSV file name : ")
    if os.path.exists(data_file)==False:
        raise Exception(f"not found {data_file}")
    else:
        print(f"Process file {data_file}")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    feq=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {feq} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()

# Read csv file and extract and transform data as need

In [66]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df.loc[from_str:to_str,['symbol','price']]
print(df.info())

list_symbol=df['symbol'].unique().tolist() 
print(list_symbol)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 297 entries, 2023-05-10 to 2023-09-29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  297 non-null    object 
 1   price   297 non-null    float64
dtypes: float64(1), object(1)
memory usage: 7.0+ KB
None
['QQQ', 'ACWI', 'THD']


In [67]:
df.head(len(list_symbol)*3)

,symbol,price
date,,
2023-05-10,QQQ,325.14
2023-05-10,ACWI,92.19
2023-05-10,THD,73.85
2023-05-11,QQQ,326.20
2023-05-11,ACWI,91.96
2023-05-11,THD,72.77
2023-05-12,QQQ,325.03
2023-05-12,ACWI,91.70
2023-05-12,THD,72.17


In [68]:
df.tail(len(list_symbol)*3)

,symbol,price
date,,
2023-09-27,QQQ,355.04
2023-09-27,THD,63.19
2023-09-27,ACWI,92.06
2023-09-28,THD,62.94
2023-09-28,ACWI,92.63
2023-09-28,QQQ,358.01
2023-09-29,QQQ,358.27
2023-09-29,ACWI,92.37
2023-09-29,THD,62.49


# Basic

# Resample data and calculate Percentage of change

In [69]:
colsROC=['symbol','date','pct_change']
df_Pct_Price=pd.DataFrame(columns=colsROC)
df_Pct_Price

,symbol,date,pct_change


In [70]:
for symbol_name in list_symbol:

    dfx_symbol=df.query('symbol==@symbol_name')
    print(f"===================================={symbol_name}====================================")

    dfx_px=dfx_symbol.resample(timeframe).last()
    dfx_px=dfx_px.reset_index()
    print(f"List {timeframe} Price")
    print(dfx_px)

    dfx_px['pct_change']=dfx_px['price'].pct_change()*100
    dfx_px['pct_change']=dfx_px['pct_change'].round(2)

    dfx_px=dfx_px.dropna() # the first  rown 
    dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
    dfx_px=dfx_px[colsROC]
    
    print(f"Calculate ROC of {symbol_name}")
    print(dfx_px)
    
    df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

====================================QQQ====================================
List M Price
        date symbol   price
0 2023-05-31    QQQ  347.99
1 2023-06-30    QQQ  369.42
2 2023-07-31    QQQ  383.68
3 2023-08-31    QQQ  377.99
4 2023-09-30    QQQ  358.27
Calculate ROC of QQQ
  symbol       date  pct_change
1    QQQ 2023-06-30        6.16
2    QQQ 2023-07-31        3.86
3    QQQ 2023-08-31       -1.48
4    QQQ 2023-09-30       -5.22
====================================ACWI====================================
List M Price
        date symbol  price
0 2023-05-31   ACWI  91.62
1 2023-06-30   ACWI  95.94
2 2023-07-31   ACWI  99.39
3 2023-08-31   ACWI  96.50
4 2023-09-30   ACWI  92.37
Calculate ROC of ACWI
  symbol       date  pct_change
1   ACWI 2023-06-30        4.72
2   ACWI 2023-07-31        3.60
3   ACWI 2023-08-31       -2.91
4   ACWI 2023-09-30       -4.28
====================================THD====================================
List M Price
        date symbol  price
0 2023-05-31

In [71]:
df_Pct_Price

,symbol,date,pct_change
0,QQQ,2023-06-30,6.16
1,QQQ,2023-07-31,3.86
2,QQQ,2023-08-31,-1.48
3,QQQ,2023-09-30,-5.22
4,ACWI,2023-06-30,4.72
5,ACWI,2023-07-31,3.60
6,ACWI,2023-08-31,-2.91
7,ACWI,2023-09-30,-4.28
8,THD,2023-06-30,-3.79
9,THD,2023-07-31,7.54


# Sum percentage of fund return

In [72]:
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[['pct_change']].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Pric=dfSum_Pct_Price.sort_values(by="pct_change",ascending=False)
dfSum_Pct_Price["pct_change"]=dfSum_Pct_Price["pct_change"].round(1)
dfSum_Pct_Price

,symbol,pct_change
0,ACWI,1.1
1,QQQ,3.3
2,THD,-8.1


In [73]:
# writer=pd.ExcelWriter( "BasicFundPerformanceReport.xlsx",engine='openpyxl') 

# df_Pct_Price.to_csv(f"ETF-{timeframe}-Return.csv",index=False)
# dfSum_Pct_Price.to_csv(f"ETF-{timeframe}-TotalReturn.csv",index=False)


# Advance

# Pivot Percent Return To Dataframe

In [75]:
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values='pct_change')
# print(dfPct.info())
dfPct

symbol,ACWI,QQQ,THD
date,,,
2023-06-30,4.72,6.16,-3.79
2023-07-31,3.60,3.86,7.54
2023-08-31,-2.91,-1.48,-2.40
2023-09-30,-4.28,-5.22,-9.49


# Take Percent Return to Find Rank

In [76]:
dfRank=dfPct[dfPct.columns.tolist()].rank(method='max',axis=1)
dfRank

symbol,ACWI,QQQ,THD
date,,,
2023-06-30,2.0,3.0,1.0
2023-07-31,1.0,2.0,3.0
2023-08-31,1.0,3.0,2.0
2023-09-30,3.0,2.0,1.0


In [77]:
# Combine Both return and Rank

In [78]:
dfSummary=pd.concat([\
                     dfPct.sum().to_frame(name='sum_return'),\
                     dfRank.sum().to_frame(name='sum_rank'),\
                    ],axis=1)
dfSummary.fillna(value=0,inplace=True)
dfSummary

,sum_return,sum_rank
symbol,,
ACWI,1.13,7.0
QQQ,3.32,10.0
THD,-8.14,7.0


In [79]:
dfFinalScore=dfSummary.rank(method='max',axis=0)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore.sort_values(by='TotalScore',ascending=False,inplace=True)
dfFinalScore

,sum_return,sum_rank,TotalScore
symbol,,,
QQQ,3.0,3.0,6.0
ACWI,2.0,2.0,4.0
THD,1.0,2.0,3.0


In [45]:
with pd.ExcelWriter(f"ETFPerReport_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
    dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
    dfSum_Pct_Price.to_excel(writer, sheet_name="Return")
    dfRank.to_excel(writer, sheet_name="RANK")
    dfSummary.to_excel(writer, sheet_name="Summary")
    dfFinalScore.to_excel(writer, sheet_name="FinalScore")